In [1]:
%load_ext autoreload
%autoreload 2
import sys, os
sys.path.append(os.path.abspath("../src"))

In [ ]:
from region_extractor import RegionWrapper, MinecraftRegionExtractor
import pandas as pd

extractor = MinecraftRegionExtractor("/home/kyre/repos/minecraft-world-generator/data/RUNETALE Converged Realms")
regions = extractor.get_regions()

df = pd.DataFrame(columns=["Region_X", "Region_Z", "Inhabited_Time"])

for region in regions:
    inhabited_time = sum(region.get_inhabited_times())
    df.add({
        "Region_X": region.x,
        "Region_Z": region.z,
        "Inhabited_Time": inhabited_time
    })

Region (-1, -1) - Total Inhabited Time: 1082933
Region (-1, -2) - Total Inhabited Time: 464481
Region (-1, 0) - Total Inhabited Time: 602061
Region (-2, -1) - Total Inhabited Time: 871727
Region (-2, -2) - Total Inhabited Time: 422344
Region (-2, 0) - Total Inhabited Time: 482929
Region (-3, -5) - Total Inhabited Time: 102974
Region (-3, -6) - Total Inhabited Time: 194014
Region (-4, -5) - Total Inhabited Time: 75574
Region (-4, -6) - Total Inhabited Time: 139355
Region (-7, -5) - Total Inhabited Time: 0
Region (0, -1) - Total Inhabited Time: 263196
Region (0, -2) - Total Inhabited Time: 112111
Region (0, 0) - Total Inhabited Time: 150635
Region (-1, -1) - Total Inhabited Time: 0
Region (-1, -5) - Total Inhabited Time: 0
Region (-1, 0) - Total Inhabited Time: 0
Region (-3, -5) - Total Inhabited Time: 92027
Region (-3, -6) - Total Inhabited Time: 282196
Region (-4, -5) - Total Inhabited Time: 33320
Region (-4, -6) - Total Inhabited Time: 100656
Region (0, -1) - Total Inhabited Time: 0
R

In [10]:
import matplotlib.pyplot as plt

# plt.hist(df["Inhabited_Time"], bins=50)
df.describe()



,Region_X,Region_Z,Inhabited_Time
count,0,0,0
unique,0,0,0
top,NaN,NaN,NaN
freq,NaN,NaN,NaN
